<a href="https://colab.research.google.com/github/quanndm/kltn/blob/dev/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# init

In [5]:
!rm -rf kltn
!rm -rf sample_data
!git clone https://github.com/quanndm/kltn.git

Cloning into 'kltn'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 23 (delta 4), reused 19 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (23/23), done.
Resolving deltas: 100% (4/4), done.


In [2]:
from kltn.init import install_dependencies
install_dependencies.install_packages("./kltn")
